This script was written for post-processing output from Allen Center segmentation directly from gridded array for ZDR column related analysis

In [1]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]
plt.rcParams["font.size"] = 13
# package for io 
from aicsimageio import AICSImage, omeTifWriter                            

# function for core algorithm
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper, dot_2d_slice_by_slice_wrapper
from aicssegmentation.core.vessel import filament_2d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from aicssegmentation.core.utils import hole_filling
from skimage.morphology import remove_small_objects, watershed, dilation, erosion, ball     # function for post-processing (size filter)
from skimage.feature import peak_local_max
from skimage.measure import label,regionprops,regionprops_table
from scipy.ndimage import distance_transform_edt
from scipy.stats import norm

# for dataframe compatibility of zdr object properties and matplotlib features for lightning plot
import pandas as pd
import glob
import skimage
from aicsimageio import AICSImage
from aicssegmentation.cli.to_analysis import simple_builder, masked_builder
from datetime import datetime 
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from datetime import timedelta
import matplotlib

import pyart
from copy import deepcopy
from skimage.external import tifffile
from natsort import natsorted
import matplotlib.pyplot as plt
import os
import xarray as xr

%load_ext autoreload
%autoreload 2


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



### Create gridded radar files after removing noise from Z$_{DR}$ field

In [32]:
#dimensions of grid in meters
x_lower = -75000
x_upper = 45000
xsize = x_upper - x_lower
y_lower = 0 
y_upper = 120000
ysize = y_upper - y_lower
frz_lvl=4000
xyresolution=500

min_height=0
max_height=15000 #TODO make max_height dynamic to make script more efficient
zresolution=500

def get_grid(radar):
    """ Returns grid object from radar object. """
    
    zdr=radar.fields['differential_reflectivity']['data']
    #remove data with rhohv<0.8
    rhohv = radar.fields['cross_correlation_ratio']['data']
    zdr_column = np.greater(zdr, 1.0)
    rhohv_low = np.less(rhohv, 0.80)
    badData = np.logical_and(zdr_column, rhohv_low)
    radar.fields['differential_reflectivity']['data'] = np.ma.masked_where(badData, zdr)
        
    display = pyart.graph.RadarMapDisplay(radar)

    filter = pyart.filters.GateFilter(radar)
    filter.exclude_below('reflectivity', 20)
    filter = pyart.correct.despeckle_field(radar, 'reflectivity', size=30, gatefilter=filter)
    corrected_reflectivity = deepcopy(radar.fields['reflectivity'])
    corrected_differential_reflectivity = deepcopy(radar.fields['differential_reflectivity'])

    corrected_reflectivity['data'] = np.ma.masked_where(filter.gate_excluded, corrected_reflectivity['data'])
    corrected_differential_reflectivity['data'] = np.ma.masked_where(filter.gate_excluded,corrected_differential_reflectivity['data'])

    radar.add_field('corrected_reflectivity', corrected_reflectivity, replace_existing=True)
    radar.add_field('corrected_differential_reflectivity', corrected_differential_reflectivity, replace_existing=True)
    
    fields = ['corrected_differential_reflectivity']#,'kdp']

    grids = pyart.map.grid_from_radars(radar, grid_shape=(int((max_height-min_height)/zresolution) +1 , int((ysize)/xyresolution) +1, 
                                                          int((xsize)/xyresolution) +1),grid_limits=((min_height, max_height),
                                                          (y_lower, y_upper), (x_lower, x_upper)), fields=fields,
                                                          roi_func='constant', gridding_algo="map_gates_to_grid",weighting_function='BARNES',
                                                          constant_roi=1149.)
    return grids

### Apply 3D segmentation on gridded Z$_{DR}$ field

In [33]:
def suggest_normalization_param_custom(structure_img0):
    m, s = norm.fit(structure_img0.flat)
#     print(f'mean intensity of the stack: {m}')
#     print(f'the standard deviation of intensity of the stack: {s}')

    p99 = np.percentile(structure_img0, 99.99)
#     print(f'0.9999 percentile of the stack intensity is: {p99}')

    pmin = structure_img0.min()
#     print(f'minimum intensity of the stack: {pmin}')

    pmax = structure_img0.max()
#     print(f'maximum intensity of the stack: {pmax}')

    up_ratio = 0
    for up_i in np.arange(0.5, 1000, 0.5):
        if m+s * up_i > p99:
            if m+s * up_i > pmax:
#                 print(f'suggested upper range is {up_i-0.5}, which is {m+s*(up_i-0.5)}')
                up_ratio = up_i-0.5
            else:
#                 print(f'suggested upper range is {up_i}, which is {m+s*up_i}')
                up_ratio = up_i
            break

    low_ratio = 0
    for low_i in np.arange(0.5, 1000, 0.5):
        if m-s*low_i < pmin:
#             print(f'suggested lower range is {low_i-0.5}, which is {m-s*(low_i-0.5)}')
            low_ratio = low_i-0.5
            break
    return low_ratio,up_ratio

In [34]:
filenames = sorted(glob.glob('/Users/sharm261/Desktop/mount/May_19_2013_all_stuff/KTLX_data/*V06'))
normal = matplotlib.colors.Normalize(vmin=1, vmax=7)
cm = matplotlib.cm.ScalarMappable(norm=normal,cmap='cubehelix')

for f in filenames:
    radar = pyart.io.read(f)
    grid = get_grid(radar)
    
    arrays = [cm.to_rgba(grid.fields['corrected_differential_reflectivity']['data'][i,:,:]) for i in range(31)]
    zdr_stack = np.stack(arrays)
    zdr_stack = np.interp(zdr_stack, (zdr_stack.min(), zdr_stack.max()), (255, 0))
    zdr_stack = zdr_stack[8:23]
    
    reader = AICSImage(zdr_stack,dims="ZYXC")
    IMG = reader.data
    struct_img0 = IMG[0,:,:,1,:].copy()
    
    ################################
    intensity_scaling_param = [39,0]
    gaussian_smoothing_sigma = 1
    ################################

    # intensity normalization
    low_ratio,up_ratio = suggest_normalization_param_custom(struct_img0)

    while ((low_ratio != intensity_scaling_param[0]) or (up_ratio != intensity_scaling_param[1])): 
        struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)
        low_ratio,up_ratio = suggest_normalization_param_custom(struct_img0)

    # smoothing with gaussian filter
    structure_img_smooth = image_smoothing_gaussian_slice_by_slice(struct_img, sigma=gaussian_smoothing_sigma)
    
    s2_param = [[1.25,0.9],[1,0.07],[1,0.01],[1.5,0.005]]
    fill_2d = True
    fill_max_size = 100000
    ################################

    bw_spot = dot_2d_slice_by_slice_wrapper(structure_img_smooth, s2_param)
#     bw_spot_fill = hole_filling(bw_spot, 80, fill_max_size, fill_2d)
    
    ################################
    ## PARAMETERS for this step ##
    f2_param = [[1.25, 0.07],[1.25,0.05]]
    fill_2d = True
    fill_max_size = 100000
    ################################

    bw_filament = filament_2d_wrapper(structure_img_smooth, f2_param)
#     bw_filament_fill = hole_filling(bw_filament, 100, fill_max_size, fill_2d)
    
    bw = np.logical_or(bw_spot, bw_filament)
    bw_fill = hole_filling(bw, 500, fill_max_size, fill_2d)
    bw_fill = np.invert(bw_fill)
    
    # watershed
    minArea = 50
    Mask = remove_small_objects(bw_fill>0, min_size=minArea, connectivity=1, in_place=False) 
    Seed = dilation(peak_local_max(struct_img,labels=label(Mask), min_distance=2, indices=False), selem=ball(1))
    Watershed_Map = -1*distance_transform_edt(bw_fill)
    seg = watershed(Watershed_Map, label(Seed), mask=Mask, watershed_line=True)
    
    ################################
    ## PARAMETERS for this step ##
    minArea = 50                      
    ################################
    
    
    seg = remove_small_objects(seg>0, min_size=minArea, connectivity=1, in_place=False)
#     seg = np.ma.masked_where(mask==False,seg)
#     np.ma.set_fill_value(seg,-999)
    
    seg  = np.swapaxes(seg,0,2)
    seg = np.invert(seg)
    seg = seg >0
    out=seg.astype(np.uint8)
    out = 1 - out
    out[out>0]=255
    out = np.rot90(out[:,:,:],axes=[1,2])

    # plt.pcolormesh(out[0,:,:])
    fsave_name = f.split('/')[-1][13:19]
    
    plt.pcolormesh(out[0,:,:])
    writer = omeTifWriter.OmeTifWriter(f'/path/to/segmentation_direct_array/{fsave_name}.tiff')
    writer.save(out)

In [35]:
segmented_files = sorted(glob.glob('/path/to/segmentation_direct_array/*.tiff'))
# plt.pcolormesh(np.rot90(cell_seg_labeled[1,:,::-1],axes=[0,1]))
for f in segmented_files:
    reader1 = AICSImage(f)
    dttt = reader1.data[0,0,:,:,:]

    cell_seg_labeled = skimage.measure.label(dttt)
    n_obj = len(regionprops(cell_seg_labeled))
    plt.pcolormesh(cell_seg_labeled[0,::-1,:])
    plt.show()

In [ ]:
# This will create a napari viewer for the entire 4D (t,x,y,z) stack created for segmented images 

import napari
from dask_image.imread import imread
from skimage import measure
stack = imread("/path/to/segmentation_direct_array/*.tiff")
from vispy.color import Colormap
import matplotlib.cm as cm
import pyart

# sicne napari has limited colormaps and we want to use our custom colormap
cmap = cm.get_cmap('pyart_HomeyerRainbow', 15)
rgb_list = []
for i in range(cmap.N):
    rgb = cmap(i)[:3]
    rgb_list.append(rgb)
rgb_list[0] = (0,0,0)

cmap = Colormap(rgb_list)

# define a function which reads only the last three dimensions since our stacked object is 4D
# concept credit: https://napari.org/tutorials/dask

def last3dims(f):
    # this is just a wrapper because the pycudadecon function
    # expects ndims==3 but our blocks will have ndim==4
    def func(array):
        return f(array[0])[None,...]
    return func
    

label = last3dims(measure.label)

labeled = stack.map_blocks(label)

with napari.gui_qt():
    napari.view_image(labeled, contrast_limits=[0,15], colormap = ('HomeyerRainbow',cmap), is_pyramid=False)